In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# import os
# os.chdir("/content/drive/MyDrive/Colab Notebooks/York/Business apps/Shmueli chapters/Ch 13 ensembles")
# !ls

# Chapter 13: Combining Models: Ensembles and Uplift Modeling

> (c) 2019 Galit Shmueli, Peter C. Bruce, Peter Gedeck 
>
> Code included in
>
> _Data Mining for Business Analytics: Concepts, Techniques, and Applications in Python_ (First Edition) 
> Galit Shmueli, Peter C. Bruce, Peter Gedeck, and Nitin R. Patel. 2019.

## Import required packages

In [3]:
!pip install dmba

     |████████████████████████████████| 11.8MB 4.7MB/s 


In [4]:
%matplotlib inline

from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from dmba import classificationSummary


no display found. Using non-interactive Agg backend


## Table 13.1, 13.2

In [5]:
bank_df = pd.read_csv('https://raw.githubusercontent.com/borisgarbuzov/Business-apps/main/Data/UniversalBank.csv')
bank_df.drop(columns=['ID', 'ZIP Code'], inplace=True)

# split into training and validation
X = bank_df.drop(columns=['PersonalLoan'])
y = bank_df['PersonalLoan']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=3)

### single tree

In [6]:
defaultTree = DecisionTreeClassifier(random_state=1)
defaultTree.fit(X_train, y_train)

classes = defaultTree.classes_
classificationSummary(y_valid, defaultTree.predict(X_valid), class_names=defaultTree.classes_)

Confusion Matrix (Accuracy 0.9825)

       Prediction
Actual    0    1
     0 1778   15
     1   20  187


### bagging

In [7]:
bagging = BaggingClassifier(DecisionTreeClassifier(random_state=1), 
                            n_estimators=100, random_state=1)
bagging.fit(X_train, y_train)

classificationSummary(y_valid, bagging.predict(X_valid), class_names=classes)

Confusion Matrix (Accuracy 0.9855)

       Prediction
Actual    0    1
     0 1781   12
     1   17  190


### boosting

In [8]:
boost = AdaBoostClassifier(DecisionTreeClassifier(random_state=1), n_estimators=100, random_state=1)
boost.fit(X_train, y_train)

classificationSummary(y_valid, boost.predict(X_valid), class_names=classes)

Confusion Matrix (Accuracy 0.9840)

       Prediction
Actual    0    1
     0 1779   14
     1   18  189


## Table 13.9

In [9]:
voter_df = pd.read_csv('https://raw.githubusercontent.com/borisgarbuzov/Business-apps/main/Data/Voter-Persuasion.csv')

# Preprocess data frame
predictors = ['AGE', 'NH_WHITE', 'COMM_PT', 'H_F1', 'REG_DAYS', 
            'PR_PELIG', 'E_PELIG', 'POLITICALC', 'MESSAGE_A']
outcome = 'MOVED_AD'

classes = list(voter_df.MOVED_AD.unique())

# Partition the data
X = voter_df[predictors]
y = voter_df[outcome]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)

# Train a random forest classifier using the training set
rfModel = RandomForestClassifier(n_estimators=100, random_state=1)
rfModel.fit(X_train, y_train)

classificationSummary(y_valid, rfModel.predict(X_valid), class_names=classes)

Confusion Matrix (Accuracy 0.6415)

       Prediction
Actual    N    Y
     N 2018  450
     Y  984  548


In [10]:
uplift_df = X_valid.copy()  # Need to create a copy to allow modifying data

uplift_df.MESSAGE_A = 1
predTreatment = rfModel.predict_proba(uplift_df)
uplift_df.MESSAGE_A = 0
predControl = rfModel.predict_proba(uplift_df)

upliftResult_df = pd.DataFrame({
    'probMessage': predTreatment[:,1],
    'probNoMessage': predControl[:,1],
    'uplift': predTreatment[:,1] - predControl[:,1],
    }, index=uplift_df.index)
print(upliftResult_df.head())

      probMessage  probNoMessage  uplift
9953         0.77           0.62    0.15
3850         0.39           0.39    0.00
4962         0.20           0.14    0.06
3886         0.86           0.62    0.24
5437         0.10           0.28   -0.18


In [11]:
print(predTreatment)

[[0.23 0.77]
 [0.61 0.39]
 [0.8  0.2 ]
 ...
 [0.56 0.44]
 [0.23 0.77]
 [0.3  0.7 ]]
